In [22]:
import os
import pandas as pd

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split

from keras.applications.vgg16 import VGG16
from keras.layers import Input, Flatten, Dense
from keras.models import Model
from keras.optimizers import Adam


if str(os.getcwdb()[-3:]).split("'")[1] != 'src':
    for _ in range(2):
        os.chdir(os.path.dirname(os.getcwdb()))


In [23]:
processed_images_path = r'data\processed\images'
df_images_data = pd.read_csv(r'data\processed\images_data_processed.csv')

df_images_data.head()


,Id,weight (carat),cut quality,color quality,clarity quality,depth (percentage),length (millimeters),width (millimeters),depth (millimeters)
0,1638147,0.55,4.0,5.0,1.0,62.553191,5.05,4.35,2.94
1,1612606,0.51,4.0,2.0,3.0,64.900662,4.71,4.35,2.94
2,1638140,0.50,4.0,2.0,3.0,62.813522,4.91,4.26,2.88
3,1536093,0.53,4.0,6.0,2.0,65.720524,4.70,4.46,3.01
4,1643527,0.52,4.0,1.0,6.0,65.141612,4.76,4.42,2.99


# Tamaño de las imágenes

- Se modifican los píxeles de cada imagen, de 300 a 224, para que puedan encajar en el modelo

In [24]:
for image_path in glob.glob(processed_images_path+'/*.jpg'):
    with Image.open(image_path) as image:
        image = image.resize((224, 224))
        image.save(image_path)


In [25]:
# Se comprueba que el cambio ha surtido efecto
for image in glob.glob(processed_images_path+'/*.jpg'):
    image_matrix = cv2.imread(image)
    break

image_matrix.shape


(224, 224, 3)

# "Split"

- Se separa el "dataframe" en "train" y "test"

In [26]:
df_images_data['Id'] = df_images_data['Id'].apply(lambda x: x + '.jpg')

df_images_data.head()


,Id,weight (carat),cut quality,color quality,clarity quality,depth (percentage),length (millimeters),width (millimeters),depth (millimeters)
0,1638147.jpg,0.55,4.0,5.0,1.0,62.553191,5.05,4.35,2.94
1,1612606.jpg,0.51,4.0,2.0,3.0,64.900662,4.71,4.35,2.94
2,1638140.jpg,0.50,4.0,2.0,3.0,62.813522,4.91,4.26,2.88
3,1536093.jpg,0.53,4.0,6.0,2.0,65.720524,4.70,4.46,3.01
4,1643527.jpg,0.52,4.0,1.0,6.0,65.141612,4.76,4.42,2.99


In [38]:
X_train, X_test, y_train, y_test = train_test_split(df_images_data['Id'], df_images_data.drop(columns='Id'), train_size=0.8, random_state=42)

df_train = pd.concat((X_train, y_train), axis=1)
df_test = pd.concat((X_test, y_test), axis=1)

df_train.head()


,Id,weight (carat),cut quality,color quality,clarity quality,depth (percentage),length (millimeters),width (millimeters),depth (millimeters)
3778,1798065.jpg,0.50,4.0,0.0,3.0,61.825319,5.08,5.11,3.15
978,1786532.jpg,0.31,4.0,5.0,5.0,51.371571,4.89,3.13,2.06
251,1634076.jpg,0.80,4.0,1.0,2.0,64.947469,5.43,5.04,3.40
2154,1643658.jpg,0.59,4.0,3.0,5.0,47.773973,7.13,4.55,2.79
4099,1769140.jpg,0.50,4.0,5.0,1.0,59.323671,5.17,5.18,3.07


# "Data augmentation"

- Se crea una variable para generar imágenes en diferentes posiciones para que el modelo disponga del mismo diamante colocado de modos distintos


In [39]:
# Se establecen las variables para crear nuevos diamantes y para seleccionar el tamaño de imagen correcto
data_augment = ImageDataGenerator(rotation_range=20,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    )


In [40]:
train_generator = data_augment.flow_from_dataframe(dataframe=df_train,
                                                        directory=processed_images_path,
                                                        target_size=(224, 224),
                                                        class_mode='raw',
                                                        shuffle=False,
                                                        x_col='Id',
                                                        y_col=list(df_images_data.columns[1:]),
                                                        )


Found 3615 validated image filenames.


# Modelaje

- Se elige VGG16 por las siguientes razones:

1) Es popular y se ha utilizado con éxito en investigación
2) Es relativamente fácil utilizarlo para problemas de regresión
3) Tiene un buen rendimiento
4) Trabaja con RGB, y el color de los diamantes es importante
5) Utiliza un tamaño de 224x224, y se ha visto que a partir de 150 componentes se obtiene toda la información necesaria

In [41]:
# Se carga el modelo sin la capa superior
base_model = VGG16(include_top=False, input_tensor=Input(shape=(224, 224, 3)))


In [42]:
# Se congelan las capas base para que no se entrenen al tunear parámetros, sino que queden igual
for layer in base_model.layers:
    layer.trainable = False
